# Poincloud rotor estimation

Consider the following challenge. We are presented with an input pointcloud $p_i$, and an output pointcloud $q_i = R[p_i] + \eta_i$, where $R$ is an unknown tranformation, and $\eta_i$ is Gaussian noise. The challenge is to reconstruct the transformation $R$.

In order to do this, we construct a symbolic tranformation $R$, whose entries are `symfit.Parameter` objects. We can then use `symfit` to find the rotor $R$.

In [1]:
from kingdon import Algebra
from symfit import Fit, Model, CallableModel, Variable, Parameter, Eq, Mul
from symfit.core.minimizers import *
import numpy as np

We set up the number of points `n_points` in the pointcloud, the number of (Euclidean) dimensions of the modeling space `d`, and the standard deviation `sig` of the Gaussian distribution.

In [2]:
n_points = 10
d = 2
sig = 0.02

In [3]:
point_vals = np.zeros((d+1, n_points))
noise_vals = np.zeros((d+1, n_points))
point_vals[-1, :] = np.ones(n_points)
point_vals[:d, :] = np.random.random((d, n_points))
noise_vals[:d, :] = np.random.normal(0.0, sig, (d, n_points))

In [4]:
alg = Algebra(d, 0, 1)
locals().update(alg.blades)

Create the points and noise as pseudovector of grade `d`.

In [5]:
noise = alg.multivector(noise_vals, grades=(d,))
p = alg.multivector(point_vals, grades=(d,))
p

[0.66291658 0.37100789 0.43365448 0.9723016  0.71834403 0.43138626
 0.79247265 0.12060475 0.28759719 0.44793232] 𝐞₀₁ + [0.65621705 0.86190161 0.55143665 0.27474101 0.60589696 0.92750396
 0.49083472 0.67982363 0.20211856 0.73123945] 𝐞₀₂ + [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 𝐞₁₂

As the input rotor $R$, we use a translation by $0.5$ in the $\mathbb{e}_{20}$ direction, followed by a rotation around $\mathbb{e}_{12}$.

In [6]:
t = np.pi/3
T = alg.multivector({'e': 1, 'e02': -0.5})
print(f'{T=!s}')
S = alg.multivector({'e': np.cos(t), 'e12': np.sin(t)})
print(f'{S=!s}')
R = S*T


T=1 + -0.5 𝐞₀₂
S=0.5 + 0.866 𝐞₁₂


We can now create the transformed pointcloud $q$, and visualize both both $p$ and $q$.

In [7]:
q = R.sw(p) + noise

In [8]:
alg.graph(0xff0000, p, 'p', 0x0000ff, q, 'q', 0x000000, R.grade(2), 'axis')

<IPython.core.display.Javascript object>

We will now setup a symfit model to describe this transformation, where the rotor $R$ consists of `Parameter`'s, and the pointclouds $p$ and $q$ are symfit `Variable`'s.

In [9]:
R_par = alg.evenmv(name='R', symbolcls=Parameter)
p_var = alg.multivector(name='p', symbolcls=Variable, grades=(d,))
q_var = alg.multivector(name='q', symbolcls=Variable, grades=(d,))
print(R_par)
print(p_var)
print(q_var)

R + R01 𝐞₀₁ + R02 𝐞₀₂ + R12 𝐞₁₂
p01 𝐞₀₁ + p02 𝐞₀₂ + p12 𝐞₁₂
q01 𝐞₀₁ + q02 𝐞₀₂ + q12 𝐞₁₂


In [10]:
p_var_trans = R_par.sw(p_var).filter()
model = Model({q_var[k]: expr for k, expr in p_var_trans.items()})
model

Prepare the data for `symfit`.

In [11]:
datadict = {p_var[k].name: p[k] for k in p_var.keys()}
datadict.update({q_var[k].name: q[k] for k in q_var.keys()})

Initiate a `symfit.Fit` object with the model and data. We additionally supply the demand $R \widetilde{R} = 1$, since rotors should be normalized (i.e., othonormal transformations).

In [12]:
constraints = [
    Eq(R_par.normsq()[0], 1)
]
fit = Fit(model, **datadict, constraints=constraints)

In [13]:
results = fit.execute()
print(results)


Parameter Value        Standard Deviation
R         5.019900e-01 1.539838e-02
R01       -4.331861e-01 1.926272e-02
R02       -2.529785e-01 1.305376e-02
R12       8.648735e-01 9.985278e-03
Status message         Optimization terminated successfully
Number of iterations   19
Objective              <symfit.core.objectives.LeastSquares object at 0x0000020EEB0ABE50>
Minimizer              <symfit.core.minimizers.SLSQP object at 0x0000020EEB0CC610>

Goodness of fit qualifiers:
chi_squared            0.007945238291911466
objective_value        0.003972619145955733
r_squared              0.9927462211035855

Constraints:
--------------------
Question: R**2 + R12**2 - 1 == 0?
Answer:   3.954787608506649e-10




`symfit` has used SLSQP because of the constraint, and we see that we have high accuracy on this constraint. Let us print the reconstructed rotor and it's norm. Furthermore, we can now apply $\widetilde{R}$ to $q$ to transform it back to the location of $p$ so we can visually inspect the quality of the reconstruction.

In [14]:
R_re = R_par(**results.params)
print(R_re)
print(R_re.normsq())

0.502 + -0.433 𝐞₀₁ + -0.253 𝐞₀₂ + 0.865 𝐞₁₂
1.0


In [15]:
p_reconstructed = (~R_re).sw(q)

In [16]:
alg.graph(0xff0000, p, 0x0000ff, q, 'q', 0x880088, p_reconstructed, 'p reconstructed', 0x000000, R_re.grade(2), 'reconst. axis')

<IPython.core.display.Javascript object>

We see that we have excellent agreement between the original and reconstructed pointclouds.